## Pemodelan prediksi IOD menggunakan LSTM

Data yang digunakan merupakan indeks IOD yang dapat diunduh melalui laman : http://www.bom.gov.au/climate/enso/indices.shtml?bookmark=iod

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
import itertools
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 1. Load dan pra-pemrosesan data
data = pd.read_excel('Data IOD.xlsx')

In [3]:
# Konversi kolom 'Tanggal' ke format datetime jika belum
data['Tanggal'] = pd.to_datetime(data['Tanggal'], errors='coerce') # Konversi ke datetime

data.set_index('Tanggal', inplace=True)  # Atur kolom tanggal sebagai indeks

In [4]:
data

,Nilai IOD
Tanggal,
2008-08-03,0.06
2008-08-10,0.10
2008-08-17,0.19
2008-08-24,-0.02
2008-08-31,-0.07
...,...
2024-09-22,-0.48
2024-09-29,-0.43
2024-10-06,-0.38


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 845 entries, 2008-08-03 to 2024-10-20
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Nilai IOD  845 non-null    float64
dtypes: float64(1)
memory usage: 13.2 KB


In [6]:
# 2. Normalisasi data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[['Nilai IOD']])

# Fungsi untuk membentuk data sequence
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Parameter sequence
sequence_length = 4  # Menggunakan 4 minggu sebelumnya untuk memprediksi minggu berikutnya
X, y = create_sequences(data_scaled, sequence_length)

In [7]:
# 3. Membagi data menjadi train dan test set
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [8]:
# 4. Membangun model LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1)),
    Dense(1)
])

# 5. Kompilasi dan pelatihan model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.1290 - val_loss: 0.0160
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0070 - val_loss: 0.0024
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0022 - val_loss: 0.0014
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0018 - val_loss: 0.0014
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0018 - val_loss: 0.0014
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0018 - val_loss: 0.0014
Epoch 9/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 10/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 11/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017 - val_loss: 0.0013
Epoch 12/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0018 - val_l

In [9]:
# Evaluasi model
train_score = model.evaluate(X_train, y_train)
test_score = model.evaluate(X_test, y_test)
print(f'MSE Data Training: {train_score}')
print(f'MSE Data Testing: {test_score}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.9931e-04
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.9880e-04 
MSE Data Training: 0.0008909585303626955
MSE Data Testing: 0.0006051347590982914


In [10]:
# 6. Prediksi Nowcasting (prediksi nilai saat ini berdasarkan data terakhir)
nowcast_input = data_scaled[-sequence_length:]
nowcast_input = np.reshape(nowcast_input, (1, sequence_length, 1))
nowcast_prediction = model.predict(nowcast_input)
nowcast_value = scaler.inverse_transform(nowcast_prediction)[0][0]
print("Nowcasting IOD saat ini:", nowcast_value)

# 7. Prediksi Forecasting (prediksi nilai beberapa langkah ke depan)
forecast_steps = 4  # contoh: prediksi 4 minggu ke depan
forecast_input = data_scaled[-sequence_length:]
forecast_output = []

for i in range(forecast_steps):
    forecast_input = np.reshape(forecast_input, (1, sequence_length, 1))
    pred = model.predict(forecast_input)
    forecast_output.append(pred[0][0])
    forecast_input = np.append(forecast_input[:, 1:, :], [[pred[0]]], axis=1)

forecast_values = scaler.inverse_transform(np.array(forecast_output).reshape(-1, 1))
print("Forecasting IOD 4 minggu ke depan:", forecast_values.flatten())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
Nowcasting IOD saat ini: -0.50154305
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
Forecasting IOD 4 minggu ke depan: [-0.50154305 -0.5089123  -0.4936528  -0.4830695 ]


In [11]:
# Replace these with the actual values for nowcasting and forecasting:
actual_nowcast_value = np.array([data_scaled])  # Single actual value for nowcasting
actual_forecast_values = np.array(data_scaled).flatten()  # Flatten to 1D for forecasting

# Nowcasting Metrics
mse_nowcast = mean_squared_error([actual_nowcast_value], [nowcast_value])
mae_nowcast = mean_absolute_error([actual_nowcast_value], [nowcast_value])
r2_nowcast = r2_score([actual_nowcast_value], [nowcast_value])

print("Nowcasting Metrics:")
print("MSE:", mse_nowcast)
print("MAE:", mae_nowcast)
print("R2 Score:", r2_nowcast)

# Forecasting Metrics
forecast_values_flat = forecast_values.flatten()  # Ensure forecast values are 1D
mse_forecast = mean_squared_error(actual_forecast_values, forecast_values_flat)
mae_forecast = mean_absolute_error(actual_forecast_values, forecast_values_flat)
r2_forecast = r2_score(actual_forecast_values, forecast_values_flat)

print("\nForecasting Metrics:")
print("MSE:", mse_forecast)
print("MAE:", mae_forecast)
print("R2 Score:", r2_forecast)

ValueError: Found array with dim 4. None expected <= 2.